# Optional: House Prices Data Exploration

In this notebook we will explore datasets in which the features are not images, but CSV files with different features, either numeric or categorical. More specifically, we will look at the housing dataset, famous for being used as an introduction to regression problems, which are problems in which we try to predict a continous variable (eg: the price of a house), rather than a discrete one (eg: the label of an image). 

The challenges of working with such a dataset are different from the ones we encounter when working with image data. Because each feature is basically the answer to one question (eg: 'Where is the house?', 'Does it have a pool?', 'What surface does the house have?' etc), some of the features might be missing or might have invalid values.

This notebook can be seen as a starting point to analyize the house prices data. We will load the data and show of some useful pandas functions for visualizing and understanding the data, as well as some transformations useful if we want to later train a model for predicting house prices. 

![alt text](https://i.pinimg.com/originals/fa/6a/74/fa6a7435b27da8fc82705f9552806235.jpg)

In [ ]:
from exercise_code.data.csv_dataset import CSVDataset
from exercise_code.data.csv_dataset import FeatureSelectorAndNormalizationTransform
from exercise_code.data.dataloader import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline
%load_ext autoreload
%autoreload 2

# 0. Loading the data

Let us get started by downloading our data. In `exercise_code/data/csv_dataset.py` you can find a class CSVDataset, which is already implemented for you, but you will understand better by following this notebook.

Similar to the last exercise, this class automatically downloads the raw data for you. To do so, simply initialize the class as below:

In [ ]:
download_url = 'https://cdn3.vision.in.tum.de/~dl4cv/housing_train.zip'
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
root_path = os.path.join(i2dl_exercises_path, "datasets", 'housing')
target_column = 'SalePrice'

In [ ]:
dataset = CSVDataset(target_column=target_column, root=root_path, download_url=download_url)

You should now be able to see a file called `housing_train.csv` in `i2dl_exercises/data/housing`.

If you wish to redownload the data, simply call:

In [ ]:
dataset = CSVDataset(target_column=target_column, root=root_path, download_url=download_url, force_download=True)

# 1. Data Exploration

We will now have a look at the dataset. Before doing so, however, we will manually load the data here, outside the `CSVDataset` class. We can simply load it by calling the Pandas function `read_csv` with the correct path of our dataset.

The entire dataset is contained in `i2dl_exercises/data/housing/housing_train.csv` which you have downloaded before.

In [ ]:
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
housing_file_path = os.path.join(i2dl_exercises_path, "datasets", "housing", "housing_train.csv")
df = pd.read_csv(housing_file_path)

Before continuing with data exploration, let's just see a few rows of the newly created `pd.DataFrame`, so that we can see what are the features of the data.

In [ ]:
# Inspect a few rows from the dataset.
df.head()

## 1.1 Statistics about the numeric features

In [ ]:
# Using the describe function we can get an overview about numerical ranges of the features.
df.describe()

## 1.2 Statistics about the target column

The housing dataset is used for trying to predict the price of a house by looking at several different characteristics of it. Thus, our target column will be the one refering to the sale price. 

In [ ]:
# Check the names of all columns so that we can find the exact name of the target column.
print(df.columns)

In [ ]:
# We can also look at the same statistics for the target variable.
target_column = 'SalePrice'
df[target_column].describe()

In [ ]:
# To better understand the price distribution, we can also look at a histogram of the prices. 
sns.distplot(df[target_column], kde=False)
plt.title("Distribution of Sale Price")
plt.ylabel("Number of Occurences")
plt.xlabel("Sale Price");

## 1.3 Looking at the relationship between a feature and the target variable

In [ ]:
# We can look at the relationship between different features and the target variable.
# For example, we can see how (and if) the pool area, overall quality, living area 
# or the year the house was built influence the price.

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))

df.plot.scatter('PoolArea', target_column, ax=axes[0][0])
df.plot.scatter('OverallQual', target_column, ax=axes[0][1])
df.plot.scatter('GrLivArea', target_column, ax=axes[1][0])
df.plot.scatter('YearBuilt', target_column, ax=axes[1][1])

## 1.4 Plotting correlations

In [ ]:
# We can also plot the correlation between all pairs of features.
# Looking at the lightly-coloured cells in the plot, we can discover 
# features that are correlated. 
#
# For example, 'GarageYrBlt' and 'YearBuilt' have high correlation.
# Also, 'GarageCars' and 'GarageArea' are highly correlated (what a surprise!)

f, ax = plt.subplots(figsize=(12, 9))

corrmat = df.corr()
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
# Saleprice correlation matrix:
# We look at the 10 most correlated variables for our target "Sale Price".
#
# This shows that there are certain attributes much more correlated than 
# the others for SalePrice.

f, ax = plt.subplots(figsize=(10, 8))
sns.set(font_scale=1.25)

k = 10 
cols = corrmat.nlargest(k, target_column)[target_column].index
cm = np.corrcoef(df[cols].values.T)
hm = sns.heatmap(cm, 
                 cbar=True, 
                 annot=True, 
                 square=True, 
                 fmt='.2f', 
                 annot_kws={'size': 10}, 
                 yticklabels=cols.values, 
                 xticklabels=cols.values)

## 1.5 Pairwise scatterplots

In [ ]:
# Next, we explore the scatter plot of selected attributes.
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.set()
sns.pairplot(df[cols], height = 2.5)

# 2.Cleaning and preparing the data

Now, that we have a better understanding of the data, we can start preparing it for the model.

## 2.1 Dealing with missing values

Values can be missing from a dataset for different reasons and the missing values can be random or not. Even if dropping the rows with missing values seems to be a tempting solution, some algorithms might not support that and you may see an error message like this:

<pre><code> ValueError: Input contains NaN, infinity or a value too large for dtype('float64'). </code></pre>

Or, if they support it, their performance might be affected, because we might unawarely introduce a bias in the model. In some situations, we might not even have access to too much data, as data collection is an expensive process, so it is too valuable too just throw away an entire row for a single missing value. Instead, we will focus on filling out the missing values.

For further explanations regarding missing values and an interactive tutorial, you can check out [this tutorial](https://www.craft.ai/blog/missing-values).

In [ ]:
# First, let's see if there are any missing values in the dataset.
df.isnull().values.any()

In [ ]:
# We can see how many null entries are for each column.
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

In [ ]:
# If one column has much more missing values than present ones, 
# then it is reasonable to completely remove that column.
max_number_of_nans = 0.5 * len(df)
df = df.loc[:, (df.isnull().sum(axis=0) <= max_number_of_nans)]


# We can fill in the missing numerical values using the mean
# for each column:
df = df.fillna(df.mean())

# Let's check if there are still missing values!
df.isnull().values.any()

In [ ]:
# There are still categorical variables with missing values.
# One simple way to fill them is to choose the value with the highest
# frequency on each column. 
null_columns = df.columns[df.isnull().any()]
for column in null_columns:
    most_frequent_value = df[column].value_counts().nlargest(n=1).index[0]
    df[column].fillna(most_frequent_value, inplace=True)

In [ ]:
# Still any null values? No, we're done!
df.isnull().values.any()

For simplicity, we only showed how to fill in the missing values using the mean for numeric fields and the mode for categorical variables. A more sophisticated metod for filling in missing values is using a machine learning algorithm (such as decision trees or K-nearest-neighbours) for predicting them, using the columns with no missing values as training data, but this will not be covered in this notebook.

## 2.3 Dealing with categorical variables

So until now we made sure there are no missing values. But we are still left with some categorical data, such as the 'LotShape' feature, which we cannot just feed into an algorithm. We need to transform these values to numerical values first. We will use a [one-hot encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) for this.

In [ ]:
# Check out possible values of a categorical column.
np.unique(df['LotShape'])

In Pandas, we can convert categorical variables to one-hot encodings simply by using the `get_dummies` function. 


For example, for the 'LotShape' column we had the values 'IR1', 'IR2', 'IR3' and 'Reg', then after the one-hot encoding process we will have 4 columns instead, 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR2' and 'LotShape_Reg' and only one of them will be 1 (depending on the original value), while the rest will be 0.

In [ ]:
df = pd.get_dummies(df)

# Key error: 'LotShape' does not exist anymore.
# df['LotShape']

# 'LotShape_IR1' has only the values 0 and 1.
np.unique(df['LotShape_IR1'])

## 2.2 Normalizing the data

Just as we did with images, we also need to normalize our data, as machine learning algorithms do not perform well when different features have different scales. If we want to normalize a feature $ x $ in the $[0, 1]$ range, then we can use the following formula:

$$ x_\text{new} = \frac{x - x_\text{min}}{x_\text{max} - x_\text{min}} $$

In Pandas, we can use the [min](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.min.html) and [max](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html) DataFrame methods in order to find the minimum and maximum value per column, for each numerical feature. 

In [ ]:
# Pandas knows how to broadcast the df.min() dataframe (which has
# just one row with the minimum value for each column) such that
# we can subtract it from df.
df = (df - df.min()) / (df.max() - df.min())

# Now all values are between 0 and 1.
df.head()

## 2.3 Implementing a transform

Until now, we have shown the usual csv dataset preprocessing steps on the raw data. The aim, however, is to encapsulate both the data and the transforms in a `CSVDataset` object and to apply the transform not on the whole data at once, but rather on each item and only when required (that is, when `CSVDataset.__getitem__` will be called for that specific row). Even though the housing prices dataset is small enough to fit in memory and apply the tranform at once on the whol dataframe, in other situations we might have to deal with much bigger datasets, so it is good practice to implement transforms that are applied on individual rows of the dataframe, especially as this is also the way in which the standard PyTorch transforms are applied.

Because in the next exercise we will use the housing dataset for training a small model, we want to implement a transform that does the following:
* Selects a few (2-3) numeric columns from the dataframe
* Normalizes these selected features between 0 and 1 
* Discards the rest of the features

In machine learning, it is always important that any preprocessing step we apply on the train data is also applied on the test and validation data. For normalization, we have seen that we need the minimum and maximum value of each numeric column. For filling in missing numeric values (if any), we need the mean value. These values we should pre-compute and feed them to the transform and for the test dataset we should also use it.

We have already implemented the transform for you, you can check it out the `FeatureSelectorAndNormalizationTransform` class in `exercise_code/data/csv_dataset.py`.

In [ ]:
# Re-read the data first.
df = pd.read_csv(housing_file_path)

In [ ]:
# Select only 2 features to keep plus the target column.
selected_columns = ['OverallQual', 'GrLivArea', target_column]
mn, mx, mean = df.min(), df.max(), df.mean()

column_stats = {}
for column in selected_columns:
    crt_col_stats = {'min' : mn[column],
                     'max' : mx[column],
                     'mean': mean[column]}
    column_stats[column] = crt_col_stats    

In [ ]:
transform = FeatureSelectorAndNormalizationTransform(column_stats, target_column)
train_dataset = CSVDataset(target_column=target_column, root=root_path, download_url=download_url, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)

Now, as already shown in the cifar10 notebook, we will be able to use the housing prices dataloader for training a model by directly accessing the batches:

In [ ]:
for batch in dataloader:
    # do some machine learning (we'll see in the next weeks how!)
    pass